In [6]:
# population_utf8.csv reading functionality
import csv
file1Data={}
n=0
with open("population_utf8.csv") as infile:
    rd = csv.reader(infile, delimiter=";")
    for line in rd:
        # Note: Only finnish name is used as a key.
        file1Data[line[0].split(';')[0]] = int(line[2])


In [7]:
# results-2019-01-nopeustesti.csv reading functionality

# Dictionary which holds all necessary information from both files
file2Data = {}
with open("results-2019-01-nopeustesti.csv",  errors="ignore") as infile:
    rd = csv.reader(infile, delimiter=",")
    next(infile)
    for line in rd:
        # Collect data only from municipalities which are present in both files.
        if line[3] in file1Data.keys():
            # Combine loading speeds to already existing municipality
            if line[3] in file2Data:
                if line[4] != 'n/a':
                    file2Data[line[3]]['downl'].append(int(line[4]))
                if line[5] != 'n/a':
                    file2Data[line[3]]['upl'].append(int(line[5]))
            # Otherwise create new municipality & initalize it
            else:
                dl = [int(line[4])] if line[4] != 'n/a' else []
                ul = [int(line[5])] if line[5] != 'n/a' else []
                file2Data[line[3]] = {'pop': file1Data[line[3]], 'downl': dl, 'upl': ul}


In [8]:
import json
import numpy as np
# Combine array with json-objects filled with asked information
resJson = []
for name in list(file2Data.keys()):
    dlavg = np.array(file2Data[name]['downl']).mean()
    dlvar = np.array(file2Data[name]['downl']).var()
    ulavg = np.array(file2Data[name]['upl']).mean()
    ulvar = np.array(file2Data[name]['upl']).var()
    
    resJson.append({'municipality': name,
                    'population': file2Data[name]['pop'],
                    'DL avg': dlavg,
                    'DL var': dlvar,
                    'UL avg': ulavg,
                    'UL var': ulvar
                   })

# Write collected data to json-file
with open("netspeeds.json", "w") as outfile:
    json.dump(resJson, outfile, ensure_ascii=False, indent = 4)

In [9]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Combine xml-object filled with asked information
nsData = ET.Element('NetSpeedData')
for key in list(file2Data.keys()):
    Municipality = ET.SubElement(nsData, 'Municipality')
    name = ET.SubElement(Municipality, 'Name')
    name.text = key
    pop = ET.SubElement(Municipality, 'Population')
    pop.text = str(file2Data[key]['pop'])
    dlavg = ET.SubElement(Municipality, 'DLavg')
    dlavg.text = str(np.array(file2Data[key]['downl']).mean())
    dlvar = ET.SubElement(Municipality, 'DLvar')
    dlvar.text = str(np.array(file2Data[key]['downl']).var())
    ulavg = ET.SubElement(Municipality, 'ULavg')
    ulavg.text = str(np.array(file2Data[key]['upl']).mean())
    ulvar = ET.SubElement(Municipality, 'ULvar')
    ulvar.text = str(np.array(file2Data[key]['upl']).var())

# Write collected data to xml-file
nsDataStr = ET.tostring(nsData).decode("latin")
with open("netspeeds.xml", "w") as outfile:
    outfile.write(minidom.parseString(nsDataStr).toprettyxml(indent="  "))
    
